<a href="https://colab.research.google.com/github/stanislavlia/DeepLearning.AI-specialization/blob/main/DeepNN_regression_california_house_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

In [3]:
train_df = pd.read_csv("sample_data/california_housing_train.csv")

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           17000 non-null  float64
 1   latitude            17000 non-null  float64
 2   housing_median_age  17000 non-null  float64
 3   total_rooms         17000 non-null  float64
 4   total_bedrooms      17000 non-null  float64
 5   population          17000 non-null  float64
 6   households          17000 non-null  float64
 7   median_income       17000 non-null  float64
 8   median_house_value  17000 non-null  float64
dtypes: float64(9)
memory usage: 1.2 MB


In [12]:
#scaling values to improve convergence
stdscaler = StandardScaler()

train_df_scaled = pd.DataFrame(stdscaler.fit_transform(train_df), columns = train_df.columns)

In [24]:
X_train = train_df_scaled.drop("median_house_value", axis=1).values.T
y_train = train_df_scaled["median_house_value"].values.reshape(1, -1)


In [36]:
#My class of DeepNN

class DeepNN():
    def __init__(self, X, Y, dims_of_layers, activations, alpha = 0.01, l2_reg = 0):
        #dims_of_layers - list of number of units in each layer (first element - num of features in input)
        #activations - activation function applied to each layer

        #dims_of_layers[0] - n of features in input
        #activations[0] - activation for first hidden layer
        #we support only 3 activation funcs: linear, sigmoid, relu

        #X.shape should be (n_features, m_examples)
        #Y.shape should be (1, m_examples)

        #regularization - regularization rate
        self.X = X
        self.Y = Y

        self.m_examples = X.shape[1]

        self.dims_of_layers = dims_of_layers
        self.n_layers = len(activations)

        self.activations = activations
        self.params = dict()

        self.learning_history = []

        self.alpha = alpha
        self.l2_reg = l2_reg

        #setting cache dicts for backpropogation

        self.cache = dict()

    def initialize_params(self):
      #Now, we are going to use He-initialization of weights

        for i in range(1, len(self.activations) + 1):

            #setting parameters layer by layer
            self.params["W" + str(i)] = np.random.randn(self.dims_of_layers[i], self.dims_of_layers[i-1])
            #multiplying by constant according to He init
            self.params["W" + str(i)] *=  np.sqrt(2 / (self.dims_of_layers[i-1]))


            self.params["b" + str(i)] = np.zeros((self.dims_of_layers[i], 1))



    def activation(self, Z, function="linear"):
        if function == "linear":
            return Z

        if function == "sigmoid":
            return 1 / (1 + np.exp(-Z))

        if function == "relu":
            return Z * (Z > 0)


    def deriv_activation(self, Z, function):
        if function == "linear":
            return 1.

        if function == "sigmoid":
            sigm_z = self.activation(Z, "sigmoid")

            return sigm_z *(1 - sigm_z)

        if function == "relu":
            return 1. * (Z > 0)



    def forward_propogation(self, X):

        #X.shape = (n_features, m_examples)

        A_prev = X
        self.cache["A0"]  = A_prev

        Z_current = np.dot(self.params["W1"], A_prev) + self.params["b1"]
        A_current = self.activation(Z_current, function=self.activations[0])

        self.cache["Z1"] = Z_current
        self.cache["A1"] = A_current

        for i in range(1, len(self.activations)):
            A_prev = A_current

            #A_prev - cache["A" + str(i)]
            Z_current = np.dot(self.params["W" + str(i+1)], A_prev) + self.params["b" + str(i+1)]
            A_current = self.activation(Z_current, function=self.activations[i])

            #keeping values in cache for backprop
            self.cache["Z" + str(i+1)] = Z_current
            self.cache["A" + str(i+1)] = A_current


        predictions = A_current

        return predictions

    def compute_reg_penalty(self):

      penalty = 0
      for l in range(1, self.n_layers + 1):
        W_l = self.params["W" + str(l)]
        penalty += np.sum(np.square(W_l))

      penalty = penalty * (self.l2_reg / (2 * self.m_examples))

      return penalty




    def compute_cost(self, predictions, cost_function):
        #leave cost func as a parameter
        #so that we can use it futher not only for classification

        if cost_function == "cross_entropy":

            #lets cut off a  tiny constant to avoid log0 problem
            epsilon = 10 ** -15

            predictions = np.clip(predictions, epsilon, 1-epsilon)

            #BinaryCrossEntropy

            cost = (self.Y * np.log(predictions) +
                    (1 - self.Y) * np.log(1 - (predictions)) ) * (-1 / self.m_examples)

            cost = np.sum(cost, axis=1, keepdims=True) #sum up the columns

        if cost_function == "mse":

            cost = np.sum(((predictions - self.Y) ** 2) * (2 / self.m_examples), axis=1, keepdims=True)


        #computing regularization penalty
        if self.l2_reg> 0:

          reg_penalty = self.compute_reg_penalty()
          cost += reg_penalty



        return cost

    def deriv_of_cost(self, predictions, cost_function):

        if cost_function == "cross_entropy":

            #avoiding division by zero
            epsilon = 10 ** -15
            predictions = np.clip(predictions, epsilon, 1-epsilon)

            dAL = (predictions - self.Y) / (predictions * (1 - predictions))

        if cost_function == "mse":
            dAL = (predictions - self.Y)

        return dAL


    def back_propogation(self, predictions, cost_func):

        L = self.n_layers

        grads_cache = dict()

        for i in range(L, 0, -1):
            if i == L:
                dA_i = self.deriv_of_cost(predictions, cost_function=cost_func)
            else:

                dA_i = np.dot(self.params["W" + str(i+1)].T, grads_cache["dZ" + str(i+1)]) #ERROR MuSt be here
                #print("i =", i)
                # print("W=", self.params["W" + str(i+1)] )
                # print("dZ_next =", grads_cache["dZ" + str(i+1)])

                # print("W = ", self.params["W" + str(i+1)].T.shape)
                # print("W shape ", )
                # print("dZ shape" + str(i),  grads_cache["dZ" + str(i+1)].shape)
                #print("dA", dA_i)


            grads_cache["dA" + str(i)] = dA_i
            activation_i = self.activations[i-1]

            Z_i = self.cache["Z" + str(i)]
            A_prev = self.cache["A" + str(i-1)]
            W_i = self.params["W" + str(i)]

            #print("Activation = ", activation_i)
            dZ_i = dA_i * self.deriv_activation(Z_i, activation_i)

            #print("Sum of dZ_i", np.sum(dZ_i))

            #computing derivs for W, b

            #L2 regularization term
            l2_term = (self.l2_reg / self.m_examples) * W_i

            dW_i = (1 / self.m_examples) * np.dot(dZ_i, A_prev.T) + l2_term
            db_i = (1/ self.m_examples) * np.sum(dZ_i, axis=1, keepdims=True)


            #storing gradients
            grads_cache["dZ" + str(i)] = dZ_i
            grads_cache["dW" + str(i)] = dW_i
            grads_cache["db" + str(i)] = db_i

        return grads_cache

    def update_params(self, grads):

        for i in range(1, self.n_layers + 1):

            #updating by Gradient Descent
            self.params["W" + str(i)] -=  self.alpha * grads["dW" + str(i)]
            self.params["b" + str(i)] -= self.alpha * grads["db" + str(i)]


    def fit(self, epochs=100, cost_func="mse", debug=False):

        #fitting process

        #initialize random params

        history = []
        self.initialize_params()


        for epoch in range(0, epochs + 1):

            predictions = self.forward_propogation(self.X)

            #computing cost function
            cost = np.round(self.compute_cost(predictions, cost_func), 6)
            history.append(cost)


            if (epoch % max(1 , (epochs // 20)) == 0):
              print("Epoch #{},  {} == {}".format(epoch, cost_func, cost))

            #computing gradients
            grads = self.back_propogation(predictions, cost_func=cost_func)
            if debug == True:
              print(grads)

            #update params using Gradient Descent
            self.update_params(grads)

        self.history = history

In [72]:
#setting up the network

layers = [X_train.shape[0], 10, 5, 1]
activations = ["relu", "relu", "linear"]

model = DeepNN(X_train, y_train, layers, activations, alpha = 0.1, l2_reg = 0.1)
model.fit(epochs=20000, cost_func="mse")

Epoch #0,  mse == [[7.550531]]
Epoch #1000,  mse == [[0.545248]]
Epoch #2000,  mse == [[0.520322]]
Epoch #3000,  mse == [[0.506265]]
Epoch #4000,  mse == [[0.499331]]
Epoch #5000,  mse == [[0.499682]]
Epoch #6000,  mse == [[0.492485]]
Epoch #7000,  mse == [[0.487453]]
Epoch #8000,  mse == [[0.485079]]
Epoch #9000,  mse == [[0.481657]]
Epoch #10000,  mse == [[0.479333]]
Epoch #11000,  mse == [[0.471976]]
Epoch #12000,  mse == [[0.468106]]
Epoch #13000,  mse == [[0.459757]]
Epoch #14000,  mse == [[0.453406]]
Epoch #15000,  mse == [[0.448572]]
Epoch #16000,  mse == [[0.446813]]
Epoch #17000,  mse == [[0.441419]]
Epoch #18000,  mse == [[0.441386]]
Epoch #19000,  mse == [[0.440426]]
Epoch #20000,  mse == [[0.439794]]


In [73]:
from sklearn.metrics import r2_score


train_predictions = model.forward_propogation(X_train)
print("R^2 score on train set = ", r2_score(y_train.T, train_predictions.T))

R^2 score on train set =  0.7801732724694097




**Uploading test data on housing in California**

(note) These datasets are available by default in Google Colab

In [74]:
test_df = pd.read_csv("sample_data/california_housing_test.csv")

test_df_scaled = pd.DataFrame(stdscaler.transform(test_df), columns = test_df.columns)

X_test = test_df_scaled.drop("median_house_value", axis=1).values.T
y_test = test_df_scaled["median_house_value"].values.reshape(1, -1)

In [75]:
predictions = model.forward_propogation(X_test)

In [76]:
print("R^2 score on test set = ", r2_score(y_test.T, predictions.T))

R^2 score on test set =  0.7455538017150225
